In [1]:
!hostnamectl

 Static hostname: DGX
       Icon name: computer-server
         Chassis: server 🖳
      Machine ID: dedf5691225742ff95b33f842b0781cc
         Boot ID: cb04df0ff1774eeda112c63867eeefdd
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 24.04.2 LTS]8;;              
          Kernel: Linux 6.8.0-53-generic
    Architecture: x86-64
 Hardware Vendor: NVIDIA
  Hardware Model: DGXA100 920-23687-2530-002
Firmware Version: 1.29
   Firmware Date: Tue 2024-08-06
    Firmware Age: 7month 2d


- download the data
- analyze it
- make data loaders for CL
- make train function for CL
- train, test model and save metrics

## imports

In [ ]:
import kagglehub
import shutil
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch
import torchvision.models as models 

# Data preprocessing

In [ ]:
# download the data

os.chdir(os.path.expanduser("~ir739wb/ilyarekun")) # change this for your machine
print(os.system("pwd"))


dst = os.path.expanduser("bc_project/data")  

# moving data to project folder
if not os.path.exists(dst):
    os.makedirs(dst)
    print("Data dir created successfully")
    
if os.listdir(dst) == []:
    path = kagglehub.dataset_download("andyczhao/covidx-cxr2")
    print("Path to dataset files:", path)
    shutil.move(path, dst)
    print("Data moved successfully")
print(os.system("pwd"))


In [ ]:
def data_loaders_centralyzed():
    os.chdir(os.path.expanduser("~ir739wb/ilyarekun/bc_project/01data/9"))

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Tensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
    ])
    
    train_dataset = ImageFolder('train', transform=transform)
    test_dataset = ImageFolder('test', transform=transform)
    val_dataset = ImageFolder('val', transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    return train_loader, test_loader, val_loader

# Centralyzed Learning

In [ ]:

model = models.resnet18(pretrained = True)

for param in model.parameters():
    param.requires_grad = False
    
num_classes = 2


# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
